In [ ]:
# make small scale datasets
path_dest_datasets_root = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop-small/'
non_makeup_txt_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/non-makeup.txt'
makeup_txt_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/makeup.txt'
with open(non_makeup_txt_path,'r') as f :
    list_non_makeup = [x.strip() for x in f.readlines()]
with open(makeup_txt_path,'r') as f :
    list_makeup = [x.strip() for x in f.readlines()]
set_train_nmk = set()
set_train_mk = set()
set_test_nmk = set()
set_test_mk = set()
import random
for train_nmk in list_non_makeup:
    if random.random() <= 0.7:
        set_train_nmk.add(train_nmk)
    else:
        set_test_nmk.add(train_nmk)
for train_mk in list_makeup:
    if random.random() <= 0.7:
        set_train_mk.add(train_mk)
    else:
        set_test_mk.add(train_mk)
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/train-non-makeup.txt','a') as f :
    for x in set_train_nmk:
        f.write(x+'\n')
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/train-makeup.txt','a') as f :
    for x in set_train_mk:
        f.write(x+'\n')
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/test-makeup.txt','a') as f :
    for x in set_test_mk:
        f.write(x+'\n')
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/test-non-makeup.txt','a') as f :
    for x in set_test_nmk:
        f.write(x+'\n')

In [ ]:
import torch
import sys
sys.path.append('..')
from mt_training.config import get_config
from fvcore.common.config import CfgNode
config = get_config()
a = CfgNode.load_yaml_with_base(config)
print(a)

In [ ]:
# make small scale datasets
path_dest_datasets_root = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/'
path_source_datasets_root = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/'
non_makeup_txt_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/non-makeup.txt'
makeup_txt_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/makeup.txt'
with open(non_makeup_txt_path,'r') as f :
    list_non_makeup = [x.strip() for x in f.readlines()]
with open(makeup_txt_path,'r') as f :
    list_makeup = [x.strip() for x in f.readlines()]
set_train_nmk = set()
set_train_mk = set()
set_test_nmk = set()
set_test_mk = set()
import random
for x in list_non_makeup:
    if ('mirror' not in x ):
        set_train_nmk.add(x)
for x in list_makeup:
    if ('mirror' not in x ):
        set_train_mk.add(x)
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/non-makeup_no_mirror.txt','a') as f :
    for x in set_train_nmk:
        f.write(x+'\n')
with open('/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/makeup_no_mirror.txt','a') as f :
    for x in set_train_mk:
        f.write(x+'\n')

In [ ]:
import os 
img_1_path = '/media/pc/LabServers/hengda/EleGANt-eg3d/EleGANt/crop_error_img_1.txt'
origin_non_makeup_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/non-makeup_origin.txt'
origin_makeup_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/makeup_origin.txt'
non_makeup = []
makeup = []
with open(img_1_path,'r') as f:
   for line in f.readlines():
        if 'non-make' in line:
            non_makeup.append(line.strip('\n'))         
        else :
            makeup.append(line.strip('\n'))
non_makeup_origin = []
makeup_origin = []
with open(origin_non_makeup_path,'r') as f :
    for line in f.readlines():
        non_makeup_origin.append(line.strip('\n'))
with open(origin_makeup_path,'r') as f :
    for line in f.readlines():
        makeup_origin.append(line.strip('\n'))
set_non_makeup = set(non_makeup) 
set_makeup = set(makeup)
set_non_makeup_origin = set(non_makeup_origin)
set_makeup_origin = set(makeup_origin)
result_non_makeup = set_non_makeup_origin - set_non_makeup
result_makeup = set_makeup_origin - set_makeup
print(len(result_non_makeup))
result_makeup_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/makeup.txt'
result_non_makeup_path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/non_makeup.txt'
with open(result_makeup_path,'a') as f:
    for x in result_makeup :
        f.write(x+'\n')
with open(result_non_makeup_path,'a') as f:
    for x in result_non_makeup :
        f.write(x+'\n')

In [ ]:
import torch
import torchvision
from skimage import io
import os, sys
import sys
sys.path.append('..')
from models.modules import tps
import numpy as np 


path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/lms/makeup/87989.npy'
path_2 = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/lms/non-makeup/xfsy_0238.npy'
a = np.load(path)
b = np.load(path_2)
pts_before_t = torch.from_numpy(a).unsqueeze(0).float()
pts_after_t = torch.from_numpy(b).unsqueeze(0).float()
print(a.min())
c = torch.flip(pts_before_t,dims=[1])
pts_before_t = pts_before_t /512
pts_after_t = pts_after_t /512
pts_before_t = torch.flip(pts_before_t,dims=[1])
pts_after_t = torch.flip(pts_after_t,dims=[1])
im = (torch.from_numpy(io.imread('/media/pc/LabServers/hengda/EleGANt-eg3d/EleGANt/vHX544.png').astype('float32'))
      .permute(2, 0, 1)
      .unsqueeze(0))/256
warp = tps.WarpTPS()
newim = warp(im, pts_before_t[0:1,48:,:], pts_after_t[0:1,48:,:])
print(newim.shape)
# # for i in range(len(pts_after[0])):
# #     newim[0,:,pts_after[0][i][0], pts_after[0][i][1]] = torch.Tensor([1,0,0])

# torchvision.utils.save_image(
#     torchvision.utils.make_grid(newim), 'result.png')

In [ ]:
# look at landmarks

import torch
path = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/lms/makeup/vHX544.npy'
path_2 = '/media/pc/LabServers/hengda/datasets/MT-Dataset-crop/lms/non-makeup/vSYYZ267.npy'
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
a = np.load(path)
b = np.load(path_2)
from scipy.ndimage import rotate


print(a.shape)
plt.scatter(a[:,0],a[:,1])
for i in range(len(a)):
    plt.annotate(i,(a[i,0],a[i,1]))
plt.show()
plt.scatter(b[:,0],b[:,1])
for i in range(len(b)):
    plt.annotate(i,(b[i,0],b[i,1]))
plt.show()


In [ ]:
# landmarks
import dlib
import numpy as np
from scipy.ndimage import rotate
def get_landmark(filepath, predictor):
    """get landmark with dlib
    :return: np.array shape=(68, 2)
    """
    detector = dlib.get_frontal_face_detector()

    img = dlib.load_rgb_image(filepath)
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        shape = predictor(img, d)

    t = list(shape.parts())
    a = []
    for tt in t:
        a.append([tt.x, tt.y])
    lm = np.array(a)
    lm = rotate(lm,90).transpose(1,0)
    return lm
model_path = '../pretrained_models/shape_predictor_68_face_landmarks.dat'
predictor = dlib.shape_predictor(
    model_path)

input_dir_type = 'non-makeup'
input_img_path = f'/media/pc/LabServers/hengda/datasets/MT-Dataset/images/{input_dir_type}'
files = os.listdir(input_img_path)
save_dir_path = f'/media/pc/LabServers/hengda/datasets/MT-Dataset/lms_crop/{input_dir_type}'
os.makedirs(save_dir_path, exist_ok=True)
for file in files:
    if '.png' in file:
        try:
            source_img_path = os.path.join(input_img_path,file)
            lm = get_landmark(source_img_path, predictor)
            np.save(os.path.join(save_dir_path,file.replace('.png','.npy')), lm)
        except:
            print('error_file is :',file)


In [ ]:
#encoder result latents  .pt-> .npy
import torch 
import numpy as np 
path = '/media/pc/LabServers/hengda/e4e_eg3d/save_encoder/inversions/latents/'
dest_path = '/media/pc/LabServers/hengda/EG3D-projector/eg3d/projector_out/encoder'
files = os.listdir(path)
for file in files:
    files_path = os.path.join(path,file)
    tmp = torch.load(files_path)
    tmp_npy = tmp.numpy()
    np.save(os.path.join(dest_path,file.replace('.pt','.npy')),tmp_npy)

import torch

In [ ]:
# get latent code by optimization

import os
path_test_imgs = '/media/pc/LabServers/hengda/EleGANt-eg3d/EleGANt/data_for_test/test_mt_optimize/'
dirs = ['epoch_89','epoch_93','epoch_94','epoch_100', 'epoch_106', 'epoch_112']
for di in dirs:
    
    files = os.listdir(os.path.join(path_test_imgs,di))
    print(len(files))
    for file in files:
        if '.png' in file:
            path_file = os.path.join(path_test_imgs,di,file)
            # cmd = f'CUDA_VISIBLE_DEVICES=0 python ./run_projector.py --image_path {path_file} --datajson /media/pc/LabServers/hengda/datasets/MT-Dataset/images/mirror_non_makeup/all_mt_dataset.json --latent_space_type w_plus --outdir=../results/optimize_out --network ../pretrained_models/ffhq512-128.pkl'
            # os.system(cmd)       
